In [1]:
# Importing the Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
print(dataset.shape)
dataset.head(n=2)

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [3]:
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [4]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_2 = LabelEncoder()
X['Gender'] = labelencoder_X_2.fit_transform(X['Gender'])
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.80,3,1,0,113931.57
3,699,France,0,39,1,0.00,2,0,0,93826.63
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10


In [6]:
X1 = pd.get_dummies(X)
X1.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1


In [7]:
X1.index

RangeIndex(start=0, stop=10000, step=1)

In [8]:
y.index

RangeIndex(start=0, stop=10000, step=1)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size = 0.2, random_state = 0)

In [10]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

((8000, 12), (2000, 12), (8000,), (2000,))


In [11]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
X_train

array([[ 0.16958176, -1.09168714, -0.46460796, ..., -1.01460667,
        -0.5698444 ,  1.74309049],
       [-2.30455945,  0.91601335,  0.30102557, ..., -1.01460667,
         1.75486502, -0.57369368],
       [-1.19119591, -1.09168714, -0.94312892, ...,  0.98560362,
        -0.5698444 , -0.57369368],
       ..., 
       [ 0.9015152 ,  0.91601335, -0.36890377, ...,  0.98560362,
        -0.5698444 , -0.57369368],
       [-0.62420521, -1.09168714, -0.08179119, ..., -1.01460667,
        -0.5698444 ,  1.74309049],
       [-0.28401079, -1.09168714,  0.87525072, ..., -1.01460667,
         1.75486502, -0.57369368]])

In [13]:
################################### LOGISTIC REGRESSION ######################################################

In [14]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
Pred_logistic = logisticRegr.predict_proba(X_test)
Pred_logistic

array([[ 0.79137257,  0.20862743],
       [ 0.65539739,  0.34460261],
       [ 0.85485434,  0.14514566],
       ..., 
       [ 0.79871111,  0.20128889],
       [ 0.87396484,  0.12603516],
       [ 0.76640491,  0.23359509]])

In [15]:
def KS_Function(y_actual, Pred_df):
    Test_df_logistic = pd.DataFrame({'actual':y_actual,'predicted':Pred_df.flatten()})
    Test_df_logistic['good'] = y_test
    Test_df_logistic['bad'] = 1-y_test
    Test_df_logistic['Rank'] = pd.qcut(Test_df_logistic['predicted'], 10,labels=False)
    Test_df_logistic
    grouped_logistic = Test_df_logistic.groupby('Rank',as_index = False)
    
    #KS Computation
    Final_df_logistic = pd.DataFrame()
    Final_df_logistic['min_scr'] = pd.DataFrame(grouped_logistic.min().predicted)
    Final_df_logistic['max_scr'] = grouped_logistic.max().predicted
    Final_df_logistic['bads'] = grouped_logistic.sum().bad
    Final_df_logistic['goods'] = grouped_logistic.sum().good
    Final_df_logistic['total'] = Final_df_logistic.bads + Final_df_logistic.goods
    
    KS_table_logistic = (Final_df_logistic.sort_index(by = 'min_scr',ascending=False)).reset_index(drop = True)
    KS_table_logistic['Cumm_goods'] = np.round((KS_table_logistic.goods.cumsum()/KS_table_logistic.goods.sum()),4)*100
    KS_table_logistic['Cumm_bads'] = np.round((KS_table_logistic.bads.cumsum()/KS_table_logistic.bads.sum()),4)*100
    KS_table_logistic['ks'] = KS_table_logistic['Cumm_goods'] - KS_table_logistic['Cumm_bads']
    return(KS_table_logistic)

In [16]:
logistic_KS = KS_Function(y_test,Pred_logistic[:,1])
logistic_KS

/var/app/anaconda/projects/pv48452/fintech_modeling_gateway2/envs/default/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


,min_scr,max_scr,bads,goods,total,Cumm_goods,Cumm_bads,ks
0,0.463269,0.904765,86,114,200,28.15,5.39,22.76
1,0.334986,0.462386,113,87,200,49.63,12.48,37.15
2,0.257092,0.334940,150,50,200,61.98,21.88,40.10
3,0.201289,0.256901,157,43,200,72.59,31.72,40.87
4,0.162075,0.200379,164,36,200,81.48,42.01,39.47
5,0.127847,0.162004,173,27,200,88.15,52.85,35.30
6,0.098208,0.127811,187,13,200,91.36,64.58,26.78
7,0.071659,0.098138,186,14,200,94.81,76.24,18.57
8,0.048802,0.071494,188,12,200,97.78,88.03,9.75
9,0.011105,0.048593,191,9,200,100.00,100.00,0.00


In [17]:
################################### ANN ######################################################

In [23]:
np.random.seed(1337)
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import initializers
import sklearn
from sklearn.grid_search import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [53]:
#initializing the ANN
classifier = Sequential()
#Adding the input layer and firsh hidden layer to the network
classifier.add(Dense(units = 6,kernel_initializer = initializers.glorot_normal(seed=5) , activation = 'relu', input_dim = 12))
#Adding the second hidden layer to the network
classifier.add(Dense(units = 6,kernel_initializer = initializers.glorot_normal(seed=5) , activation = 'relu'))
#Adding ouput layer
classifier.add(Dense(units = 1,kernel_initializer = initializers.glorot_normal(seed=5) , activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [54]:
# Fitting classifier to the Training set
classifier.fit(X_train,y_train,batch_size = 100 , epochs = 200, verbose=2, shuffle=False) #verbose : 0 - silent ; 1 - progress bar; 2 - one line per epoch 

Epoch 1/200
 - 1s - loss: 0.5494 - acc: 0.7589
Epoch 2/200
 - 0s - loss: 0.4996 - acc: 0.7946
Epoch 3/200
 - 0s - loss: 0.4714 - acc: 0.8030
Epoch 4/200
 - 0s - loss: 0.4535 - acc: 0.8099
Epoch 5/200
 - 0s - loss: 0.4415 - acc: 0.8111
Epoch 6/200
 - 0s - loss: 0.4330 - acc: 0.8151
Epoch 7/200
 - 0s - loss: 0.4270 - acc: 0.8189
Epoch 8/200
 - 0s - loss: 0.4225 - acc: 0.8202
Epoch 9/200
 - 0s - loss: 0.4188 - acc: 0.8229
Epoch 10/200
 - 0s - loss: 0.4156 - acc: 0.8246
Epoch 11/200
 - 0s - loss: 0.4128 - acc: 0.8266
Epoch 12/200
 - 0s - loss: 0.4101 - acc: 0.8280
Epoch 13/200
 - 0s - loss: 0.4076 - acc: 0.8299
Epoch 14/200
 - 0s - loss: 0.4054 - acc: 0.8312
Epoch 15/200
 - 0s - loss: 0.4034 - acc: 0.8319
Epoch 16/200
 - 0s - loss: 0.4014 - acc: 0.8319
Epoch 17/200
 - 0s - loss: 0.3994 - acc: 0.8335
Epoch 18/200
 - 0s - loss: 0.3975 - acc: 0.8329
Epoch 19/200
 - 0s - loss: 0.3958 - acc: 0.8336
Epoch 20/200
 - 0s - loss: 0.3939 - acc: 0.8337
Epoch 21/200
 - 0s - loss: 0.3920 - acc: 0.8337
E

In [55]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred

array([[ 0.33876225],
       [ 0.31650552],
       [ 0.12743823],
       ..., 
       [ 0.11877648],
       [ 0.18244874],
       [ 0.24008349]], dtype=float32)

In [56]:
ANN_KS = KS_Function(y_test,y_pred)
ANN_KS

/var/app/anaconda/projects/pv48452/fintech_modeling_gateway2/envs/default/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


,min_scr,max_scr,bads,goods,total,Cumm_goods,Cumm_bads,ks
0,0.652367,0.972254,32,168,200,41.48,2.01,39.47
1,0.387408,0.649758,112,88,200,63.21,9.03,54.18
2,0.246927,0.386273,144,56,200,77.04,18.06,58.98
3,0.148019,0.245934,171,29,200,84.20,28.78,55.42
4,0.098904,0.147899,172,28,200,91.11,39.56,51.55
5,0.066182,0.098136,182,18,200,95.56,50.97,44.59
6,0.045046,0.066084,196,4,200,96.54,63.26,33.28
7,0.028332,0.044651,190,10,200,99.01,75.17,23.84
8,0.011329,0.028275,199,1,200,99.26,87.65,11.61
9,0.001737,0.011320,197,3,200,100.00,100.00,0.00


In [57]:
pd.DataFrame(classifier.get_weights()).to_csv('classifier_weights.csv')

In [39]:
X_test[0:1]

array([[-0.55204276, -1.09168714, -0.36890377,  1.04473698,  0.8793029 ,
        -0.92159124,  0.64259497,  0.9687384 ,  1.61085707, -1.01460667,
         1.75486502, -0.57369368]])

In [62]:
y_pred[0:1]

array([[ 0.33876225]], dtype=float32)

In [58]:
################################### XGBoost ######################################################

In [59]:
from xgboost import XGBClassifier
xgbc = XGBClassifier(n_estimators=100, max_depth=5, learning_rate = 0.1)
xgbc.fit(X_train,y_train)
xgbc_pred=xgbc.predict_proba(X_test)
xgbc_pred

array([[ 0.6757229 ,  0.3242771 ],
       [ 0.79776293,  0.20223708],
       [ 0.86877739,  0.13122258],
       ..., 
       [ 0.91349173,  0.0865083 ],
       [ 0.80397975,  0.19602026],
       [ 0.83111155,  0.16888846]], dtype=float32)

In [60]:
XGB_KS = KS_Function(y_test,xgbc_pred[:,1])
XGB_KS

/var/app/anaconda/projects/pv48452/fintech_modeling_gateway2/envs/default/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


,min_scr,max_scr,bads,goods,total,Cumm_goods,Cumm_bads,ks
0,0.654819,0.994040,31,169,200,41.73,1.94,39.79
1,0.363185,0.652215,118,82,200,61.98,9.34,52.64
2,0.231306,0.363085,146,54,200,75.31,18.50,56.81
3,0.152981,0.230166,170,30,200,82.72,29.15,53.57
4,0.104188,0.152930,168,32,200,90.62,39.69,50.93
5,0.072536,0.104166,186,14,200,94.07,51.35,42.72
6,0.051722,0.071939,190,10,200,96.54,63.26,33.28
7,0.035267,0.051720,193,7,200,98.27,75.36,22.91
8,0.020029,0.035029,195,5,200,99.51,87.59,11.92
9,0.004532,0.019957,198,2,200,100.00,100.00,0.00
